<a href="https://colab.research.google.com/github/seq2193/so-vits-svc-colab/blob/main/sovits4_0_%26_V2_%E6%8E%A8%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始项目：https://github.com/innnky/so-vits-svc

修改日期：2023年3月17日

In [ ]:
#@title 克隆的github仓库
#@markdown ##选择要克隆的github仓库分支
Clone = "4.0" #@param ["4.0","4.0-v2"]
if Clone == "4.0":
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
elif Clone == "4.0-v2":
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0-v2

In [ ]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install --upgrade pip setuptools numpy numba
#!pip install numpy==1.23.5
!pip install pyworld praat-parselmouth fairseq 
!pip install --upgrade gdown

In [ ]:
#@title 下载必要模型文件
# 源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)
# 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
# 由于源网盘无法提供http直链，根据mit协议，对模型进行二次分发，提供下载直链
#!wget -P hubert/ https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
#!curl -L https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr -o /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt
!wget -O hubert/checkpoint_best_legacy_500.pt https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr

## 加载歌手模型

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 复制Google云端硬盘模型
#@markdown 若使用，自行更改文件夹名、模型名
model_path = '/content/drive/MyDrive/44k/G_68000.pth' #@param {type:"string"}
config_path = "/content/drive/MyDrive/44k/config.json" #@param {type:"string"}
#@markdown 聚类音色泄漏控制模型（可选）
cluster_model_path = "/content/drive/MyDrive/44k/kmeans_10000.pt" #@param {type:"string"}
!cp {model_path} /content/so-vits-svc/logs/44k
!cp {config_path} /content/so-vits-svc/configs

In [ ]:
#@title 外链其他 gdown 模型
#训练中歌手模型 "spk:teto" 962條 4.0
!gdown 1-e5htroe-xO-eDQlSVb6ApD3M4g8_aLP  -O /content/so-vits-svc/logs/44k/G_68000.pth
!gdown 1-W71C4uJGEqqkHCPdQPbHM8BuaH6XO_p  -O /content/so-vits-svc/configs/config.json
!gdown 1y_g8Eb1bVrVKsWKH6km1xqHP-y3go7c1  -O /content/so-vits-svc/logs/44k/kmeans_10000.pt #聚类音色泄漏控制模型（可选）

In [ ]:
#@title 从huggingface下载模型
pth_link="https://huggingface.co/spaces/innnky/nyaru4.0/resolve/main/logs/44k/nyaru_G_126400.pth" #@param{type:"string"}
config_link="https://huggingface.co/spaces/innnky/nyaru4.0/resolve/main/logs/44k/nyaru_G_126400.pth" #@param{type:"string"}
cluster_model_path="https://huggingface.co/spaces/innnky/nyaru4.0/resolve/main/logs/44k/kmeans_10000.pt" #@param{type:"string"}
!wget -P /content/so-vits-svc/logs/44k {pth_link}
!wget -P /content/so-vits-svc/configs {config_link}
!wget -P /content/so-vits-svc/logs/44k {cluster_model_path}

# 声音转换

In [ ]:
#@title 上传.wav音频文件 
import os
from google.colab import files

save_dir = "/content/so-vits-svc/raw"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")


In [ ]:
#@markdown 下载示例音频合成 風になる
!gdown --id 1d-npX0KOYW5AE2fh0z1mkX3nFYvyrRii -O /content/so-vits-svc/raw/風になる.wav

In [ ]:
#@title 从Dropbox下载Vocal WAV
#@markdown Share_Link 中 www 改成dl 并且.wav后面删除 ?dl=0
wav_link="https://dl.dropbox.com/s/mh5yfihdmk1cnri/1_07.%20%E3%83%A4%E3%82%AD%E3%83%A2%E3%83%81_%28Vocals%29.wav" #@param{type:"string"}

!wget -P /content/so-vits-svc/raw {wav_link}

# 推理

In [ ]:
#@title 合成音频（推理）修改
#@markdown 需要将音频上传到so-vits-svc/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称
model_path = "logs/44k/G_68000.pth" #@param {type:"string"}
config_path = "configs/config.json" #@param {type:"string"}
clean_names = "1_07. \u30E4\u30AD\u30E2\u30C1_(Vocals).wav" #@param {type:"string"}
spk_list = "teto" #@param {type:"string"}
trans = "0"  #@param {type:"string"}
#@markdown **聚类音色泄漏控制**

#@markdown 0为完全不使用聚类，1为只使用聚类，通常设置0.5即可
cluster_infer_ratio = "0"  #@param {type:"string"}
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}
wav_output = "/content/so-vits-svc/results/" + clean_names + "_" + trans + "key" + "_" + spk_list + "." + wav_format

!python inference_main.py -m {model_path} -c {config_path} -n "{clean_names}" -t {trans} -s {spk_list}  -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}

#@markdown 如果您不想從此處下載，請取消選中此選項。
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)